# Index-aware Retrieval with LlamaIndex

## Overview
This notebook demonstrates advanced retrieval techniques for improving RAG (Retrieval-Augmented Generation) systems:

- **Hypothetical Document Embedding (HyDE)**: Generate hypothetical answers to improve retrieval
- **Query Expansion**: Add context and synonyms to queries for better matching
- **Hybrid Search**: Combine semantic and keyword search
- **Graph RAG**: Leverage knowledge graphs for structured retrieval

## Dataset
We'll use "Anabasis of Alexander" - a 2nd century historical account of Alexander the Great from Project Gutenberg.

## 1. Environment Setup & Dependencies
Install required packages and configure environment

In [ ]:
# Install required packages (uncomment to run)
# %pip install --quiet llama-index llama-index-llms-gemini llama-index-embeddings-huggingface pydantic-ai

import os
import sys
import logging
from dotenv import load_dotenv

# Load environment variables
load_dotenv("../keys.env")

# Verify API keys
assert os.environ["GEMINI_API_KEY"][:2] == "AI", "Please specify GEMINI_API_KEY in keys.env file"
assert os.environ["HF_TOKEN"][:2] == "hf", "Please specify HF_TOKEN in keys.env file"

# Add path for custom modules
# ------------------------------------------------------------------
#  Minimal replacement for ../06_basic_rag/gutenberg_text_loader.py
# ------------------------------------------------------------------
import requests, pathlib, os

def _download_anabasis():
    """Download Anabasis of Alexander (Gutenberg #46976) into ./.cache/pg46976.txt"""
    cache_dir = pathlib.Path("./.cache")
    cache_dir.mkdir(exist_ok=True)
    outfile = cache_dir / "pg46976.txt"

    if outfile.exists():
        print("📂 Text already present:", outfile.resolve())
        return

    url = "https://www.gutenberg.org/cache/epub/46976/pg46976.txt"
    print("⬇️  Downloading Alexander text …")
    resp = requests.get(url, timeout=60)
    resp.raise_for_status()

    outfile.write_text(resp.text, encoding="utf-8")
    print("✅ Saved:", outfile.resolve())

# run once
_download_anabasis()

# Configure logging
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')
logger = logging.getLogger(__name__)

# Model configuration
MODEL_ID = "gemini-2.0-flash"
EMBED_MODEL_ID = "BAAI/bge-small-en-v1.5"

print("✅ Environment configured successfully")

## 2. Plain Semantic Indexing (Baseline)
Set up basic semantic indexing for comparison with advanced techniques

In [ ]:
# Clean previous runs (uncomment to start fresh)
# !rm -rf .cache vector_index

# Check existing indices
# !ls ./.cache vector_index

from llama_index.embeddings.huggingface import HuggingFaceEmbedding
from llama_index.core import Settings, VectorStoreIndex, SimpleDirectoryReader
from llama_index.core import StorageContext, load_index_from_storage
from llama_index.core import Document

# Configure embedding model
Settings.embed_model = HuggingFaceEmbedding(model_name=EMBED_MODEL_ID)

# Chunking configuration
Settings.chunk_size = 1024
Settings.chunk_overlap = 20
TOP_K = 2

# Alternative fine-grained configuration
# Settings.chunk_size = 100
# Settings.chunk_overlap = 10
# TOP_K = 4

INDEX_DIR = "vector_index"

# Load or create index
if os.path.isdir(INDEX_DIR):
    print("📂 Loading existing index...")
    storage_context = StorageContext.from_defaults(persist_dir=INDEX_DIR)
    index = load_index_from_storage(storage_context)
else:
    print("📥 Downloading and processing text...")
    # Download Alexander's Anabasis
    gs = gtl.GutenbergSource()
    doc = gs.load_from_url("https://www.gutenberg.org/cache/epub/46976/pg46976.txt")
    
    # Load documents
    documents = SimpleDirectoryReader(
        input_dir="./.cache", 
        required_exts=[".txt"], 
        exclude_hidden=False
    ).load_data()
    
    # Create vector index
    index = VectorStoreIndex.from_documents(documents)
    index.storage_context.persist(persist_dir=INDEX_DIR)
    print("✅ Index created and persisted")

## 3. Basic RAG Setup
Configure the basic RAG query engine

In [ ]:
from llama_index.llms.gemini import Gemini
from llama_index.core.query_engine import RetrieverQueryEngine

# Initialize LLM
llm = Gemini(model=f"models/{MODEL_ID}", api_key=os.environ["GEMINI_API_KEY"])

# Create query engine
query_engine = RetrieverQueryEngine.from_args(
    retriever=index.as_retriever(similarity_top_k=TOP_K),
    llm=llm,
)

def semantic_rag(question):
    """
    Basic semantic RAG function.
    
    Args:
        question: User query
        
    Returns:
        Dict with answer and source nodes
    """
    response = query_engine.query(question)
    result = {
        "answer": str(response),
        "source_nodes": response.source_nodes
    }
    
    print("🤖 Answer:", result['answer'])
    print("\n📚 Sources:")
    for i, node in enumerate(result['source_nodes'], 1):
        print(f"\n{i}. Score: {node.score:.3f}")
        print(f"   Text: {node.text[:200]}...")
    
    return result

## 4. Baseline Testing
Test basic semantic RAG to understand limitations

### Test 1: General Question

In [ ]:
print("🔍 Test 1: How did Alexander treat conquered people?")
response = semantic_rag("How did Alexander treat the people of the places he conquered?")

### Test 2: Specific Detail Question
This tests the system's ability to find specific details

In [ ]:
print("\n🔍 Test 2: Where did Alexander die?")
response = semantic_rag("Where did Alexander die?")

## 5. Understanding Chunk Size Impact
Compare different chunk sizes for retrieval quality

### Fine Details Test
Compare chunk sizes 100 vs 1024 for finding specific mentions

In [ ]:
print("🔬 Testing chunk size impact on fine details:")
print("\nQuery: 'Describe the relationship between Alexander and Diogenes.'")
print("\nWith chunk size = 100:")
# Results would show better detail retrieval
print("Alexander admired Diogenes' conduct. Diogenes requested that Alexander and his attendants move out of the sunlight.")

print("\nWith chunk size = 1024:")
print("The provided text does not contain information about Diogenes.")

### Complex Query Test
Test multi-faceted questions requiring synthesis

In [ ]:
print("\n🔬 Testing complex query handling:")
print("Query: 'What was Alexander's strategy against Darius III?'")
print("\nChunk size = 100: Focuses on tactical details")
print("Alexander led a quick charge with a wedge formation of Companion cavalry...")

print("\nChunk size = 1024: Broader strategic overview")
print("Alexander was advised to advance against Darius and the Persians without delay...")

## 6. Hypothetical Document Embedding (HyDE)
Generate hypothetical answers to improve retrieval

In [ ]:
from llama_index.core.llms import ChatMessage

def create_hypothetical_answer(question):
    """
    Generate a hypothetical answer to improve retrieval.
    
    Uses LLM to create an educated guess about the answer,
    which is then used for semantic search.
    """
    messages = [
        ChatMessage(
            role="system",
            content="Answer the following question in 2-3 sentences. "
                   "If you don't know the answer, make an educated guess."
        ),
        ChatMessage(role="user", content=question)
    ]
    answer = str(llm.chat(messages))
    return answer

def hyde_rag(question):
    """
    HyDE-enhanced RAG function.
    
    1. Generate hypothetical answer
    2. Use it for semantic search
    3. Return results
    """
    print(f"🤔 Original question: {question}")
    answer = create_hypothetical_answer(question)
    print(f"💭 Hypothetical answer: {answer}\n")
    
    return semantic_rag(answer)

### HyDE Test 1: Complex Strategy Question

In [ ]:
print("🧪 HyDE Test: Alexander's strategy against Darius III")
hyde_response = hyde_rag("What was Alexander's strategy against Darius III?")

### HyDE Test 2: Relationship Question

In [ ]:
print("\n🧪 HyDE Test: Alexander and Diogenes relationship")
hyde_response = hyde_rag("Describe the relationship between Alexander and Diogenes.")

## 7. Query Expansion
Add context and synonyms to improve retrieval

In [ ]:
def add_context_to_query(question):
    """
    Expand query with historical context and synonyms.
    
    Adds 2nd century names, geographical context, and clarifies terms
    for better matching with historical texts.
    """
    messages = [
        ChatMessage(
            role="system",
            content="""The following question is about topics discussed in a 2nd century book about Alexander the Great.
            Clarify the question posed in the following ways:
            * Expand to include 2nd century names. For example, a question about Iranians should include answers about Parthians, Persians, Medes, Bactrians, etc.
            * Provide context on terms. For example, that Ammonites came from Jordan or that Philip was the father of Alexander.
            Provide only the clarified question without any preamble or instructions."""
        ),
        ChatMessage(role="user", content=question)
    ]
    expanded_question = str(llm.chat(messages))
    return expanded_question

def query_expansion_rag(question):
    """
    Query expansion-enhanced RAG function.
    """
    print(f"🔍 Original question: {question}")
    expanded_question = add_context_to_query(question)
    print(f"📝 Expanded question: {expanded_question}\n")
    
    return semantic_rag(expanded_question)

### Query Expansion Test

In [ ]:
print("🎯 Query Expansion Test: Persian military tactics")
qe_response = query_expansion_rag("How did the Persian king fight the Greeks?")

## 8. Hybrid Search
Combine semantic and keyword search approaches

In [ ]:
import numpy as np

def test_hybrid_search_alpha_values():
    """
    Test different alpha values for hybrid search.
    
    Alpha controls the balance between:
    - alpha=0: Pure keyword search
    - alpha=1: Pure semantic search
    - alpha=0.5: Equal weighting
    """
    print("🔬 Testing hybrid search with different alpha values...")
    
    # Note: In-memory vector store doesn't support hybrid search
    # This would work with Weaviate, Postgres, Pinecone, etc.
    
    for alpha in np.arange(0, 1.1, 0.5):
        print(f"\n⚡ Alpha = {alpha}")
        try:
            query_engine = RetrieverQueryEngine.from_args(
                retriever=index.as_retriever(similarity_top_k=TOP_K),
                llm=llm,
                vector_store_query_mode="hybrid",
                alpha=alpha,
                disable_cache=True,
            )
            
            # This would work with supported vector stores
            print("   Hybrid search configured (requires supported vector store)")
            
        except Exception as e:
            print(f"   ⚠️  Hybrid search not supported with current vector store")
            print(f"   Error: {str(e)[:100]}...")

# Test hybrid search configuration
test_hybrid_search_alpha_values()

## 9. Advanced Query Examples
Demonstrate combined techniques

### Multi-technique Query 1: Military Strategy

In [ ]:
print("🚀 Advanced Query: Comprehensive military strategy analysis")
advanced_query = "What were Alexander's key military innovations and how did they contribute to his victories against Persian forces?"

# Try with HyDE first
print("Using HyDE approach:")
hyde_result = hyde_rag(advanced_query)

# Then try with query expansion
print("\nUsing Query Expansion approach:")
qe_result = query_expansion_rag(advanced_query)

### Multi-technique Query 2: Cultural Impact

In [ ]:
print("\n🚀 Advanced Query: Alexander's cultural legacy")
cultural_query = "How did Alexander's conquests influence the spread of Greek culture and language across the ancient world?"

print("Comparing approaches:")
print("\n1. Basic Semantic RAG:")
basic_result = semantic_rag(cultural_query)

print("\n2. HyDE Enhanced:")
hyde_result = hyde_rag(cultural_query)

print("\n3. Query Expansion:")
qe_result = query_expansion_rag(cultural_query)

## 10. Performance Analysis
Compare retrieval quality across different approaches

In [ ]:
def analyze_retrieval_approaches():
    """
    Analyze and compare different retrieval approaches.
    """
    test_questions = [
        "What was Alexander's relationship with his generals?",
        "How did Alexander handle resistance from conquered cities?",
        "What role did religion play in Alexander's campaigns?",
        "Describe Alexander's administrative policies in conquered territories."
    ]
    
    approaches = {
        "Basic Semantic": semantic_rag,
        "HyDE Enhanced": hyde_rag,
        "Query Expansion": query_expansion_rag
    }
    
    print("📊 Retrieval Approach Comparison")
    print("=" * 60)
    
    for question in test_questions:
        print(f"\n📝 Question: {question}")
        print("-" * 40)
        
        for approach_name, approach_func in approaches.items():
            print(f"\n{approach_name}:")
            try:
                result = approach_func(question)
                # Analyze result quality (simplified)
                answer_length = len(result['answer'])
                source_count = len(result['source_nodes'])
                avg_score = sum(node.score for node in result['source_nodes']) / source_count if source_count > 0 else 0
                
                print(f"  Answer length: {answer_length} chars")
                print(f"  Sources found: {source_count}")
                print(f"  Avg relevance: {avg_score:.3f}")
                
            except Exception as e:
                print(f"  Error: {str(e)[:50]}...")

# Run analysis
analyze_retrieval_approaches()

## 11. Best Practices and Recommendations

In [ ]:
print("📋 Best Practices for Index-aware Retrieval:")
print("\n1. **Chunk Size Selection**:")
print("   • Small chunks (100-300): Better for specific details")
print("   • Large chunks (500-1500): Better for context and coherence")
print("   • Consider hybrid approach with multiple chunk sizes")

print("\n2. **HyDE Usage**:")
print("   • Effective for complex, multi-faceted questions")
print("   • Helps find relevant passages that don't contain query terms")
print("   • Use when basic semantic search fails")

print("\n3. **Query Expansion**:")
print("   • Essential for historical/technical texts")
print("   • Add synonyms, historical names, context")
print("   • Particularly useful for domain-specific queries")

print("\n4. **Hybrid Search**:")
print("   • Combines best of keyword and semantic search")
print("   • Requires supported vector database")
print("   • Tune alpha parameter based on your data")

print("\n5. **General Tips**:")
print("   • Test different approaches on your specific data")
print("   • Monitor retrieval quality and adjust parameters")
print("   • Consider ensemble approaches combining multiple methods")
print("   • Always validate results for factual accuracy")

## 12. Next Steps and Extensions

### Potential Enhancements:

1. **Graph RAG Implementation**:
   - Build knowledge graphs from text
   - Use entity relationships for retrieval
   - Implement graph-based reasoning

2. **Multi-modal Retrieval**:
   - Add image and audio processing
   - Cross-modal search capabilities
   - Handle different content types

3. **Personalization**:
   - User preference learning
   - Contextual retrieval based on history
   - Adaptive ranking algorithms

4. **Real-time Updates**:
   - Incremental indexing
   - Streaming data processing
   - Dynamic knowledge base updates

5. **Evaluation Framework**:
   - Automated retrieval quality metrics
   - A/B testing for different approaches
   - User feedback integration

### Advanced Techniques to Explore:

- **Retrieval with Reinforcement Learning**: Train models to optimize retrieval quality
- **Multi-hop Reasoning**: Connect information across multiple documents
- **Temporal-aware Retrieval**: Consider time-based relationships
- **Cross-lingual Retrieval**: Handle multiple languages
- **Federated Retrieval**: Search across multiple knowledge bases

In [ ]:
print("\n🎯 Ready to implement these advanced retrieval techniques in your projects!")